## Dependencies


In [2]:
import pandas as pd
import numpy as np
import requests

## Read & clean data from Excel


In [ ]:
# read_excel function
median_age = pd.read_excel('Data/median_age.xlsx', engine='openpyxl')

# use 'type' function to see what we're working with
type(median_age)

# use 'info' & 'describe' methods to find out about the dataframe
median_age.info()
median_age.describe()

# use 'astype' to change the Census Tract data type
median_age['Census Tract'] = median_age['Census Tract'].astype('str')

# remove the decimal from Census Tract column
median_age['Census Tract'] = median_age['Census Tract'].str.replace('.', '')

# show histogram
median_age.hist(bins=20)

## Read & clean data from CSV


#### Scrub the data


In [ ]:
# read_csv function
migration_df = pd.read_csv(
    'Data/countyinflow2122.csv',
    usecols=['y2_countyfips', 'y2_statefips',
             'y1_countyfips', 'y1_statefips', 'n2', 'agi']
)

# because the y1, y2 designations are confusing, replace these with 'origin' and 'dest'
new_columns = {
    col: col.replace('y1', 'origin', 1).replace('y2', 'dest', 1)
    for col in migration_df.columns
}
migration_df = migration_df.rename(columns=new_columns)

# because 'agi' is in thousands, multiply column by 1,000
migration_df['agi'] = migration_df['agi'] * 1000

# rename 'n2' column
migration_df = migration_df.rename(columns={'n2': 'persons'})

# create full FIPS codes for origin, destination
migration_df['dest_FIPS'] = migration_df['dest_statefips'].astype(
    str).str.zfill(2) + migration_df['dest_countyfips'].astype(str).str.zfill(3)
migration_df['orig_FIPS'] = migration_df['origin_statefips'].astype(
    str).str.zfill(2) + migration_df['origin_countyfips'].astype(str).str.zfill(3)

# reduce the number of columns we need
migration_df = migration_df[[
    'orig_FIPS',
    'dest_FIPS',
    'persons',
    'agi'
]]

# how to just show Fulton County?
migration_df[migration_df['orig_FIPS'] == '13121']

#### Lookup county names from FIPS code


In [2]:
# read in CSV to use as dataframe lookup table
lookup_df = pd.read_csv('Data/FIPS_lookup.csv', dtype={'FIPS': 'str'})

# ensure FIPS codes are properly zero-padded
lookup_df['FIPS'] = lookup_df['FIPS'].str.zfill(5)

# Merge to get origin county names
migration_2 = migration_df.merge(
    lookup_df[['FIPS', 'County_name', 'State']],
    how='left',
    left_on='orig_FIPS',
    right_on='FIPS'
).rename(columns={'County_name': 'orig_county', 'State': 'orig_state'})

# Drop the extra FIPS column
migration_2.drop(columns=['FIPS'], inplace=True)

# Merge to get destination county names
migration_final = migration_2.merge(
    lookup_df[['FIPS', 'County_name', 'State']],
    how='left',
    left_on='dest_FIPS',
    right_on='FIPS'
).rename(columns={'County_name': 'dest_county', 'State': 'dest_state'})

# Drop the extra FIPS column
migration_final.drop(columns=['FIPS'], inplace=True)

# rearrange the columns
migration_final = migration_final[[
    'orig_FIPS',
    'orig_county',
    'orig_state',
    'persons',
    'agi',
    'dest_FIPS',
    'dest_county',
    'dest_state'
]]

migration_final.sample(10)

NameError: name 'pd' is not defined

#### Get metro Atlanta counties


In [ ]:
# read in 29 counties from Wikipedia
url = "https://en.wikipedia.org/wiki/Metro_Atlanta"

# use 'read_html' to get the data
atl_df = pd.read_html(url)

atl_df[4]

#### Clean metro county list


In [ ]:
# pare down the dataframe
atl_df_clean = atl_df[4][[
    'County'
]]

# remove the last row, which totals the counties
atl_df_clean = atl_df_clean[atl_df_clean['County'] != 'Total']

# remove the " *" from the 'County' column
atl_df_clean['County'] = atl_df_clean['County'].str.replace(
    ' *', '', regex=False)

# add the state
atl_df_clean['State'] = 'Georgia'

# add the FIPS code
fips_df = pd.read_csv('Data/FIPS_lookup.csv')
fips_df['FIPS'] = fips_df['FIPS'].astype(str).str.zfill(5)

# merge the ATL dataframe with the FIPS lookup table
atl_df_clean = atl_df_clean.merge(
    fips_df,
    how='left',
    left_on=['County', 'State'],
    right_on=['County_name', 'State'],
).drop(columns='County_name')

# convert the FIPS values to a list
atl_fips = list(atl_df_clean['FIPS'])
atl_fips

#### Only get migration into / out of metro ATL counties


In [ ]:
# filter the migration dataset to only include ATL counties
migration_atl = migration_final[migration_final['orig_FIPS'].isin(atl_fips) |
                                migration_final['dest_FIPS'].isin(atl_fips)]

# print out rows in the dataframe
print(migration_atl.shape[0])
migration_atl.sample(10)

## Real-world example


In [4]:
fips_dictionary = {
    '01': 'Alabama',
    '02': 'Alaska',
    '04': 'Arizona',
    '05': 'Arkansas',
    '06': 'California',
    '08': 'Colorado',
    '09': 'Connecticut',
    '10': 'Delaware',
    '11': 'District of Columbia',
    '12': 'Florida',
    '13': 'Georgia',
    '15': 'Hawaii',
    '16': 'Idaho',
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '20': 'Kansas',
    '21': 'Kentucky',
    '22': 'Louisiana',
    '23': 'Maine',
    '24': 'Maryland',
    '25': 'Massachusetts',
    '26': 'Michigan',
    '27': 'Minnesota',
    '28': 'Mississippi',
    '29': 'Missouri',
    '30': 'Montana',
    '31': 'Nebraska',
    '32': 'Nevada',
    '33': 'New Hampshire',
    '34': 'New Jersey',
    '35': 'New Mexico',
    '36': 'New York',
    '37': 'North Carolina',
    '38': 'North Dakota',
    '39': 'Ohio',
    '40': 'Oklahoma',
    '41': 'Oregon',
    '42': 'Pennsylvania',
    '44': 'Rhode Island',
    '45': 'South Carolina',
    '46': 'South Dakota',
    '47': 'Tennessee',
    '48': 'Texas',
    '49': 'Utah',
    '50': 'Vermont',
    '51': 'Virginia',
    '53': 'Washington',
    '54': 'West Virginia',
    '55': 'Wisconsin',
    '56': 'Wyoming'
}

url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt'

# Fetch the content from the URL
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

table = response.text.split('------------    --------------\n')[1]

# Strip leading/trailing whitespace and split by newline
lines = table.strip().split('\n')

# Create a DataFrame from the list of lines
df = pd.DataFrame(lines, columns=['Data'])

# Split the 'Data' column on the first space
df[['FIPS', 'County_name']] = df['Data'].str.split(n=1, expand=True)

# Drop the original 'Data' column
df = df.drop(columns=['Data'])

# Drop rows where 'FIPS' ends with '000'
df = df[~df['FIPS'].str.endswith('000')]

# Extract the first 2 digits from 'FIPS' column
df['State_code'] = df['FIPS'].str[:2]

# Map 'State_code' to 'State' using fips_dict
df['State'] = df['State_code'].map(fips_dictionary)

# just get the county
df['County_short'] = df['County_name'].apply(lambda x: x.split(' County')[0])

# Drop the 'State_code' column if not needed
df_FIPS = df.drop(columns=['State_code'])

df_Georgia = df_FIPS[df_FIPS['State'] == 'Georgia']

df_FIPS

,FIPS,County_name,State,County_short
1,01001,Autauga County,Alabama,Autauga
2,01003,Baldwin County,Alabama,Baldwin
3,01005,Barbour County,Alabama,Barbour
4,01007,Bibb County,Alabama,Bibb
5,01009,Blount County,Alabama,Blount
...,...,...,...,...
3190,56037,Sweetwater County,Wyoming,Sweetwater
3191,56039,Teton County,Wyoming,Teton
3192,56041,Uinta County,Wyoming,Uinta
3193,56043,Washakie County,Wyoming,Washakie


## Pandas rapid fire


#### 1st dataset


In [ ]:
# make the dataset
data = {
    'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
    'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
    'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
    'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']
}

df = pd.DataFrame(data)

df

In [ ]:
# select rows where the age is missing
df[df['age'].isnull()]

In [ ]:
# Select the rows the age is between 2 and 4 (inclusive)
df[(df['age'] >= 2) & (df['age'] <= 4)]

# df[df['age'].between(2, 4)]

In [ ]:
# Explore the 'nunique', 'unique', and 'value_counts' methods for the 'animal' column
df['animal'].value_counts()

In [ ]:
# For each unique animal, calculate the median age
df.groupby('animal')['age'].mean()

In [ ]:
# Sort df first by the values in the 'age' in decending order, then by the value in the 'visits' column in ascending order
df.sort_values(by=['age', 'visits'], ascending=[False, True])

#### 2nd dataset


In [ ]:
# Which column of the following numbers has the largest, smallest sum?
data = {
    'A': [45, 67, 23, 89, 12],
    'B': [78, 34, 56, 90, 21],
    'C': [98, 23, 54, 65, 76],
    'D': [12, 34, 87, 45, 68],
    'E': [34, 89, 76, 54, 32],
    'F': [56, 78, 90, 12, 34]
}

df = pd.DataFrame(data)

df.sum()
df.sum().sort_values(ascending=False)
df.min().idxmax()

In [ ]:
# For each group, find the sum of the three greatest values
df = pd.DataFrame({
    'groups': list('aaabbcaabcccbbc'),
    'vals': [12, 345, 3, 1, 45, 14, 4, 52, 54, 23, 235, 21, 57, 3, 87]
})

# step 1
df.groupby('groups')['vals'].nlargest(3)

# step 3
# df.groupby('groups')['vals'].apply(lambda x: x.nlargest(3).sum())